# <big>Hands-on: Hepatocellular Carcinoma</big>

### Date: 17 October, 2019

### Author: _[Yueh-Chou Lee](https://yuehchou.github.io/)_

### This course is for National Health Insurance Administration(NHI).

---
# Target

## To classify whether the tumor will recur in the year after receiving treatment

---
# Workflow
<br>
<img src='./../include/workflow.png'>

---
# Part 0. Import necessary libraries

## 1. Check whether libraries exist

In [ ]:
!pip install numpy --user
!pip install nipype==1.2.2 --user
!pip install matplotlib --user
!pip install SimpleITK --user
!pip install xlrd --user
!pip install pandas --user
!pip install tqdm --user
!pip install pyradiomics==2.0.0 --user

## After completing the above part, you should click " _Kernel -> Restart_ " to restart the juypter notebook.

## 2. Import libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import SimpleITK as sitk
import matplotlib.pyplot as plt
from radiomics import featureextractor # version: 2.0.0 - 2.1.0

In [ ]:
params = os.path.join(os.getcwd(), './../include/radiomics_setting.yaml')
extractor = featureextractor.RadiomicsFeaturesExtractor(params)

---
# Part 1. Get the data information

## 1. NIfTI

## Unzip data file

In [ ]:
!unzip ./../data/NIfTI.zip -d ./../data/

### Get header information

In [ ]:
reader = sitk.ImageFileReader()

reader.SetFileName('./../data/NIfTI/image.nii')
reader.LoadPrivateTagsOn();

reader.ReadImageInformation();

for key in reader.GetMetaDataKeys():
    value = reader.GetMetaData(key)
    print("({0}) = = \"{1}\"".format(key, value))

### Load NIfTI image and label

In [ ]:
sitk_image = sitk.ReadImage('./../data/NIfTI/image.nii')
sitk_label = sitk.ReadImage('./../data/NIfTI/label.nii')
sitk_label = sitk.Cast(sitk_label, sitk.sitkInt8)

### Check the important information

In [ ]:
print('The physical origin point of sitk_image:', sitk_image.GetOrigin())
print('The shape of sitk_image:', sitk_image.GetSize())
print('The physical spacing of sitk_image:', sitk_image.GetSpacing())
print('The axis direction of sitk_image:', sitk_image.GetDirection())

### Transfer sitk image to numpy array

In [ ]:
np_image = sitk.GetArrayFromImage(sitk_image)
np_label = sitk.GetArrayFromImage(sitk_label)

### Check the shape, what's your observation?

In [ ]:
print('The shape of sitk_image:', sitk_image.GetSize())
print('The shape of sitk_label:', sitk_label.GetSize())
print('The shape of np_image:', np_image.shape)
print('The shape of np_label:', np_label.shape)

## 2. NRRD

In [ ]:
nrrd_image = sitk.ReadImage('./../data/NRRD/image.nrrd')

print('The physical origin point of nrrd_image:', nrrd_image.GetOrigin())
print('The shape of nrrd_image:', nrrd_image.GetSize())
print('The physical spacing of nrrd_image:', nrrd_image.GetSpacing())
print('The axis direction of nrrd_image:', nrrd_image.GetDirection())

## 3. DICOM

In [ ]:
dicom_image = sitk.ReadImage('./../data/DICOM/image.dcm')

print('The physical origin point of dicom_image:', dicom_image.GetOrigin())
print('The shape of dicom_image:', dicom_image.GetSize())
print('The physical spacing of dicom_image:', dicom_image.GetSpacing())
print('The axis direction of dicom_image:', dicom_image.GetDirection())

## 4. Choose one pair of image and label, we will continue to use this data to compute following tasks

In [ ]:
image = sitk.ReadImage('./../data/NIfTI/image.nii')
label = sitk.ReadImage('./../data/NIfTI/label.nii')
label = sitk.Cast(label, sitk.sitkInt8)

## 5. Plot the slice of this image

In [ ]:
plt.imshow(sitk.GetArrayFromImage(image)[50,:,:])

---
# Part 2. Standardize data

## 1. N4 bias field correction (for MR image)

In [ ]:
def N4ITKCorrection(Image, numberFittingLevels, numberFittingIterations):
    corrector=sitk.N4BiasFieldCorrectionImageFilter()
    corrector.SetMaximumNumberOfIterations(numberFittingIterations*numberFittingLevels)
    Image=sitk.Cast(Image,sitk.sitkFloat64)
    Image=corrector.Execute(Image)

    return Image

### How to use?

In [ ]:
numberFittingLevels=4
numberFittingIterations=[1]
N4ITK_image = N4ITKCorrection(image, numberFittingLevels, numberFittingIterations)

## 2. Simple ITK resample 

In [ ]:
def SITKResampleImage(Image, orig_spacing, NewSpacing, isLabel=False):
    orig_size = np.array(Image.GetSize(), dtype=np.int)

    NewSize = orig_size*(np.array(orig_spacing)/np.array(NewSpacing))
    NewSize = np.ceil(NewSize).astype(np.int)
    NewSize = [int(s) for s in NewSize]
    NewSize_Center = np.array(np.array(NewSize) / 2.0)

    Dimension = Image.GetDimension()
    Reference_Origin = np.zeros(Dimension)
    Transform = sitk.AffineTransform(Dimension)
    Transform.SetMatrix(Image.GetDirection())
    Transform.SetTranslation(np.array(Image.GetOrigin()) - Reference_Origin)

    Centering_Transform = sitk.TranslationTransform(Dimension)
    Image_Center = np.array(Image.TransformContinuousIndexToPhysicalPoint(np.array(Image.GetSize()) / 2.0))
    Centering_Transform.SetOffset(np.array(Transform.GetInverse().TransformPoint(Image_Center) - NewSize_Center))
    Centered_Transform = sitk.Transform(Transform)
    Centered_Transform.AddTransform(Centering_Transform)
    
    Resample = sitk.ResampleImageFilter()
    if isLabel:
        Resample.SetInterpolator = sitk.sitkNearestNeighbor
    else:
        Resample.SetInterpolator = sitk.sitkBSpline
    Resample.SetTransform(Centered_Transform)
    Resample.SetOutputDirection = Image.GetDirection()
    Resample.SetOutputOrigin = Image.GetOrigin()
    Resample.SetOutputSpacing(NewSpacing)
    Resample.SetSize(NewSize)
    ResampleImage = Resample.Execute(Image)

    return ResampleImage

### How to use?

In [ ]:
print('The spacing of N4ITK_image:', N4ITK_image.GetSpacing())
print('The spacing of label:', label.GetSpacing())

orig_image_spacing = N4ITK_image.GetSpacing()
orig_label_spacing = label.GetSpacing()
NewSpacing = ((1., 1., 5.))

Res_image = SITKResampleImage(image, orig_image_spacing, NewSpacing, isLabel=False)
Res_label = SITKResampleImage(label, orig_label_spacing, NewSpacing, isLabel=True)

print('The spacing of Res_image:', Res_image.GetSpacing())
print('The spacing of Res_label:', Res_label.GetSpacing())

## 3. Simple ITK normalize

In [ ]:
def NormalizeBasedOnLabel(Image,Label,MappingMax,MappingMin):

    RadiomicsFeatures=extractor.execute(Image,Label)

    resample_max = RadiomicsFeatures['original_firstorder_Maximum']
    resample_min = RadiomicsFeatures['original_firstorder_Minimum']

    Maxminfilter = sitk.MinimumMaximumImageFilter()
    Maxminfilter.Execute(Image)
    origin_max = Maxminfilter.GetMaximum()
    origin_min = Maxminfilter.GetMinimum()

    Scale = (MappingMax-MappingMin)/(resample_max-resample_min)
    outputMaximum = MappingMax+(origin_max-resample_max)*Scale
    outputMinimum = MappingMin+(origin_min-resample_min)*Scale

    if outputMinimum < 0:
        outputMaximum = outputMaximum - outputMinimum
        outputMinimum = 0

    NormalizedImage = sitk.RescaleIntensity(image1=Image,outputMinimum=outputMinimum,outputMaximum=outputMaximum)

    return NormalizedImage

### How to use?

In [ ]:
Maxminfilter = sitk.MinimumMaximumImageFilter()
Maxminfilter.Execute(Res_image)
origin_max = Maxminfilter.GetMaximum()
origin_min = Maxminfilter.GetMinimum()

print('Maximum of Res_image:', origin_max)
print('Minimum of Res_image:', origin_min)

MappingMax = 1024
MappingMin = 0
Norm_image = NormalizeBasedOnLabel( Res_image, Res_label, MappingMax, MappingMin)

Maxminfilter.Execute(Norm_image)
new_max = Maxminfilter.GetMaximum()
new_min = Maxminfilter.GetMinimum()

print('Maximum of Res_image:', new_max)
print('Minimum of Res_image:', new_min)

---
# Part 3. Radiomics

## 1. You can change the setting in yaml file  " _radiomics_setting.yaml_ ".

## Let's check the things in this file!

In [ ]:
!cat ./../include/radiomics_setting.yaml

## Extracting Radiomics features is time-wasted!

## So there will only show that how to extract feature for one case

## 2. Start extracting Radiomics features

In [ ]:
Result = extractor.execute(image, label)

### What's the results?

In [ ]:
for Key, Value in Result.items():
    print(Key, ':', Value)

## 3. How to save features?

### Use pandas to save these results, you can refer the code in the python file " _src/ICC.py_ ".

---
# Part 4. ICC

## 1. What's ICC?

### In statistics, the intraclass correlation, or the intraclass correlation coefficient (ICC), is a descriptive statistic that can be used when quantitative measurements are made on units that are organized into groups. 

### If ICC value is &nbsp;  (1) Below 0.50: poor &nbsp; (2) Between 0.50 and 0.75: moderate &nbsp; (3) Between 0.75 and 0.90: good &nbsp; (4) Above 0.90: excellent

### More information and formula: [Link](https://en.wikipedia.org/wiki/Intraclass_correlation)

## 2. Start to compute ICC

In [ ]:
!python ./../src/ICC.py ./../result/ICC/ICC.xlsx ./../result/features/features_file1.xlsx ./../result/features/features_file2.xlsx

---
# Part 5. Classification

## Due to the limiltation of time, we will skip to do this part!

# Part 6. Connect all above parts?

## You can try to complete this part!

In [ ]:
# ===== START ANSWER HERE =====


# ===== END ANSWER =====

# End